## Import

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

## Pyspark Session

In [4]:
spark = SparkSession.builder \
    .appName("TeslaDeliveriesAnalysis") \
    .getOrCreate()

## Dataset

In [6]:
df = spark.read.csv("./datasets/tesla_deliveries_dataset_2015_2025.csv", header=True, inferSchema=True)
df.cache()
df.show(5)
df.printSchema()

+----+-----+-------------+-------+--------------------+----------------+-------------+--------------------+--------+--------------+--------------------+-----------------+
|Year|Month|       Region|  Model|Estimated_Deliveries|Production_Units|Avg_Price_USD|Battery_Capacity_kWh|Range_km|CO2_Saved_tons|         Source_Type|Charging_Stations|
+----+-----+-------------+-------+--------------------+----------------+-------------+--------------------+--------+--------------+--------------------+-----------------+
|2023|    5|       Europe|Model S|               17646|           17922|     92874.27|                 120|     704|       1863.42|Interpolated (Month)|            12207|
|2015|    2|         Asia|Model X|                3797|            4164|     62205.65|                  75|     438|        249.46|  Official (Quarter)|             7640|
|2019|    1|North America|Model X|                8411|            9189|    117887.32|                  82|     480|        605.59|Interpolated (

## Analytics

In [21]:
# 1. Time-Series: Year-Month Level Trends + 12-Month Moving Average

df2 = df.withColumn("Date", to_date(expr("concat(Year,'-',Month,'-01')")))
window_12 = Window.orderBy("Date").rowsBetween(-11, 0)

ts = df2.groupBy("Date").agg(
    sum("Estimated_Deliveries").alias("Monthly_Deliveries")
).withColumn(
    "Rolling_12M_Avg", avg("Monthly_Deliveries").over(window_12)
).orderBy("Date")
ts.show(20, False)

+----------+------------------+------------------+
|Date      |Monthly_Deliveries|Rolling_12M_Avg   |
+----------+------------------+------------------+
|2015-01-01|183180            |183180.0          |
|2015-02-01|165053            |174116.5          |
|2015-03-01|184567            |177600.0          |
|2015-04-01|225623            |189605.75         |
|2015-05-01|184264            |188537.4          |
|2015-06-01|198787            |190245.66666666666|
|2015-07-01|177759            |188461.85714285713|
|2015-08-01|214223            |191682.0          |
|2015-09-01|188448            |191322.66666666666|
|2015-10-01|221541            |194344.5          |
|2015-11-01|211490            |195903.18181818182|
|2015-12-01|220332            |197938.91666666666|
|2016-01-01|191069            |198596.33333333334|
|2016-02-01|218600            |203058.58333333334|
|2016-03-01|191208            |203612.0          |
|2016-04-01|184134            |200154.58333333334|
|2016-05-01|224220            |

In [8]:
# 2. Anomaly Detection: Deliveries Outlier Detection (Z-Score)

stats = df.select(
    mean("Estimated_Deliveries").alias("mean_del"),
    stddev("Estimated_Deliveries").alias("std_del")
).collect()[0]

mean_del, std_del = stats

outliers = df.withColumn(
    "Zscore", (col("Estimated_Deliveries") - mean_del) / std_del
).filter(
    abs(col("Zscore")) > 3  # Strong anomaly
)

outliers.show()

+----+-----+-------------+-------+--------------------+----------------+-------------+--------------------+--------+--------------+--------------------+-----------------+------------------+
|Year|Month|       Region|  Model|Estimated_Deliveries|Production_Units|Avg_Price_USD|Battery_Capacity_kWh|Range_km|CO2_Saved_tons|         Source_Type|Charging_Stations|            Zscore|
+----+-----+-------------+-------+--------------------+----------------+-------------+--------------------+--------+--------------+--------------------+-----------------+------------------+
|2015|    3|         Asia|Model 3|               25410|           28802|     73659.66|                  60|     341|       1299.72|  Official (Quarter)|             4127|3.9349585269234173|
|2015|    6|  Middle East|Model Y|               22315|           22673|     81318.87|                  82|     472|        1579.9|Interpolated (Month)|             5909|  3.14861725553746|
|2017|   12|North America|Model 3|               2

In [23]:
# 3. EV Efficiency Index: Range per kWh (Model-Level Efficiency Ranking)

from pyspark.sql import Window
from pyspark.sql.functions import mean, stddev, col

w = Window.partitionBy("Model")

df_model_z = df.withColumn("mean_del", mean("Estimated_Deliveries").over(w)) \
               .withColumn("std_del", stddev("Estimated_Deliveries").over(w)) \
               .withColumn("Zscore", (col("Estimated_Deliveries") - col("mean_del")) / col("std_del")) \
               .filter(abs(col("Zscore")) > 3)

df_model_z.show()

+----+-----+-------------+-------+--------------------+----------------+-------------+--------------------+--------+--------------+--------------------+-----------------+------------------+------------------+------------------+
|Year|Month|       Region|  Model|Estimated_Deliveries|Production_Units|Avg_Price_USD|Battery_Capacity_kWh|Range_km|CO2_Saved_tons|         Source_Type|Charging_Stations|          mean_del|           std_del|            Zscore|
+----+-----+-------------+-------+--------------------+----------------+-------------+--------------------+--------+--------------+--------------------+-----------------+------------------+------------------+------------------+
|2015|    3|         Asia|Model 3|               25410|           28802|     73659.66|                  60|     341|       1299.72|  Official (Quarter)|             4127|10079.392045454546|3880.1182237721755|3.9510672279571253|
|2017|   12|North America|Model 3|               25704|           28939|     61247.57|  

In [12]:
# 4. Production–Delivery Supply Chain Stress Index

supply_chain = df.groupBy("Year").agg(
    sum("Production_Units").alias("Prod"),
    sum("Estimated_Deliveries").alias("Del")
).withColumn(
    "Stress_Index", (col("Prod") - col("Del")) / col("Del")
).orderBy("Year")

supply_chain.show()

+----+-------+-------+-------------------+
|Year|   Prod|    Del|       Stress_Index|
+----+-------+-------+-------------------+
|2015|2545695|2375267|0.07175109156149603|
|2016|2617000|2434081|0.07514910144732243|
|2017|2527084|2350517|0.07511836757615452|
|2018|2598455|2423104|0.07236627070072106|
|2019|2573522|2404230|0.07041422825603208|
|2020|2462234|2280336|0.07976806926698522|
|2021|2506114|2335116|  0.073228910255422|
|2022|2616834|2434220|0.07501951343756932|
|2023|2562319|2382375|0.07553135001836403|
|2024|2587664|2406899|0.07510286056872349|
|2025|2534516|2368462|0.07011047675664629|
+----+-------+-------+-------------------+



In [13]:
# 5. CO₂ Savings Intensity: CO₂ Saved per Car Delivered (Environmental KPI)

env_index = df.groupBy("Model").agg(
    (sum("CO2_Saved_tons") / sum("Estimated_Deliveries")).alias("CO2_per_Car"),
    avg("Range_km").alias("Avg_Range")
).orderBy(desc("CO2_per_Car"))

env_index.show()

+----------+-------------------+------------------+
|     Model|        CO2_per_Car|         Avg_Range|
+----------+-------------------+------------------+
|   Model 3|0.07569098853251986|503.22159090909093|
|   Model X|0.07522671235001668| 500.7765151515151|
|   Model S|0.07484062571730463| 501.2234848484849|
|Cybertruck| 0.0748091453661694| 499.8276515151515|
|   Model Y|0.07437876103959128| 496.2386363636364|
+----------+-------------------+------------------+



In [14]:
# 6. Forecasting Future Deliveries (2026–2027) using Linear Regression

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

yearly = df.groupBy("Year").agg(sum("Estimated_Deliveries").alias("Deliveries"))

assembler = VectorAssembler(inputCols=["Year"], outputCol="features")
train_df = assembler.transform(yearly)

lr = LinearRegression(featuresCol="features", labelCol="Deliveries")
model = lr.fit(train_df)

pred_df = spark.createDataFrame([(2026,), (2027,)], ["Year"])
pred = model.transform(assembler.transform(pred_df))

pred.show()

+----+--------+-----------------+
|Year|features|       prediction|
+----+--------+-----------------+
|2026|[2026.0]|2376197.309090884|
|2027|[2027.0]|2375342.209090879|
+----+--------+-----------------+



In [15]:
# 7. Price Elasticity Proxy: Does Higher Price Reduce Deliveries? (Correlation Test)
elasticity = df.stat.corr("Avg_Price_USD", "Estimated_Deliveries")
print("Price–Demand Correlation:", elasticity)

Price–Demand Correlation: -0.02754583970952005


In [16]:
# 8. Market Segmentation by Clustering (Deliveries, Range, Battery, Price)

from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StandardScaler, VectorAssembler

vec = VectorAssembler(
    inputCols=["Estimated_Deliveries", "Battery_Capacity_kWh", "Range_km", "Avg_Price_USD"],
    outputCol="features_raw"
).transform(df)

scaler = StandardScaler(inputCol="features_raw", outputCol="features", withMean=True, withStd=True)
scaled = scaler.fit(vec).transform(vec)

kmeans = KMeans(k=4, seed=42)
km_model = kmeans.fit(scaled)

clusters = km_model.transform(scaled)
clusters.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  791|
|         3|  813|
|         2|  495|
|         0|  541|
+----------+-----+



In [17]:
# 9. Regional Performance Index (Weighted Composite Score)

region_perf = df.groupBy("Region").agg(
    sum("Estimated_Deliveries").alias("Deliveries"),
    sum("CO2_Saved_tons").alias("CO2"),
    avg("Range_km").alias("Avg_Range")
).withColumn(
    "Score", col("Deliveries")*0.5 + col("CO2")*0.3 + col("Avg_Range")*0.2
).orderBy(desc("Score"))

region_perf.show()

+-------------+----------+------------------+------------------+------------------+
|       Region|Deliveries|               CO2|         Avg_Range|             Score|
+-------------+----------+------------------+------------------+------------------+
|  Middle East|   6698045|503797.13999999984| 502.5439393939394| 3500262.150787879|
|         Asia|   6539935|         484357.26|495.49848484848485|3415373.7776969695|
|       Europe|   6494035|485135.78999999916|499.66212121212124|3392658.1694242423|
|North America|   6462592| 491073.0599999999|503.32575757575756| 3378718.583151515|
+-------------+----------+------------------+------------------+------------------+



In [19]:
# 10. Model Lifecycle Analysis: How Each Model Evolves Over Time

w = Window.partitionBy("Model").orderBy("Year")

lifecycle = df.groupBy("Model", "Year").agg(
    sum("Estimated_Deliveries").alias("Del"),
    sum("Production_Units").alias("Prod")
).withColumn(
    "YoY_Del_Growth", (col("Del") - lag("Del").over(w)) / lag("Del").over(w)
).withColumn(
    "YoY_Prod_Growth", (col("Prod") - lag("Prod").over(w)) / lag("Prod").over(w)
).orderBy("Model", "Year")

lifecycle.show()

+----------+----+------+------+--------------------+--------------------+
|     Model|Year|   Del|  Prod|      YoY_Del_Growth|     YoY_Prod_Growth|
+----------+----+------+------+--------------------+--------------------+
|Cybertruck|2015|430202|461503|                NULL|                NULL|
|Cybertruck|2016|475256|511311| 0.10472754659439054| 0.10792562561890172|
|Cybertruck|2017|492785|526623| 0.03688327974817782| 0.02994654916479403|
|Cybertruck|2018|426255|459831| -0.1350081678622523|-0.12683076888020461|
|Cybertruck|2019|461653|496903| 0.08304418716495994| 0.08062092377416921|
|Cybertruck|2020|467716|505209| 0.01313324076741622| 0.01671553603017088|
|Cybertruck|2021|481289|515640|0.029019747025973028|0.020646900589656954|
|Cybertruck|2022|436080|466058| -0.0939331669745205|-0.09615623303079668|
|Cybertruck|2023|454379|492338| 0.04196248394789947| 0.05638783155744564|
|Cybertruck|2024|512794|547690| 0.12856007870082023| 0.11242682872335671|
|Cybertruck|2025|471370|503965|-0.0807